In [67]:
import numpy as np
import random
import pulp

def generate_state(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd):
    """
    生成一个系统的随机状态，该系统负责将任务分配给员工。
    
    参数:
    I_citys (int): 城市的数量。
    L_levels (int): 等级的数量。
    W_workdays (int): 员工距离放假的最大工作天数, 一般为7。
    M_servers (int): 员工的数量。
    x_max_task_num (int): 每个城市的最大任务数(本函数中未直接使用)。
    H_home_of_server (list[int]): 代表每个员工家所在城市的列表。
    lambd (np.array): 一个二维数组 (I_citys x_max_task_num L_levels)，代表每个城市和等级的任务到达率。

    返回:
    tuple: 包含任务分布矩阵和员工状态列表的元组。

    城市中的任务矩阵，员工状态列表[(员工所在地，距离放假时间)]
    """

    # 生成任务分布矩阵 (n_il)，其维度为 I_citys x_max_task_num L_levels  
    n_il = np.zeros((I_citys, L_levels), dtype=int)  # 用零初始化矩阵
    for i_city in range(I_citys):  # 遍历城市
        for l_level in range(L_levels):  # 遍历等级
            # 为每个城市和等级分配一个基于泊松分布的随机数, 表示随机状态生成的任务数量？# TODO
            n_il[i_city, l_level] = np.random.poisson(lambd[i_city, l_level])
    S0_tasks = n_il  # 任务分布矩阵 (I_citys x_max_task_num L_levels), [0, +∞)
    
    # 初始化一个列表来保存每个员工的状态，给
    S1_servers = []
    for m_server in range(M_servers):  # 遍历所有员工

        # 为员工 'm_server' 随机选择距离放假的工作日数，取值范围 [0, W_workdays]
        w_m = np.random.randint(0, W_workdays + 1)

        # 根据距离放假天数w_m，得到位置i_m，如果距离放假0，则在家里，否则随机一个位置。
        # i_m [1, I_citys]
        if w_m == W_workdays:  # 如果距离放假时间为 0 天，即今天放假，则员工所在城市为家所在城市
            i_m = H_home_of_server[m_server]
        else:  # 否则，为员工随机选择一 个非家乡城市工作
            i_m = np.random.randint(1, I_citys + 1)
            
        # 将员工的状态作为元组（城市，距离放假的工作日数）添加到列表中
        S1_servers.append((i_m, w_m))
    
    # 将任务分布矩阵和员工状态列表合并成一个状态元组
    S = (S0_tasks, S1_servers)    # ((I_citys x_max_task_num L_levels), (M_servers x_max_task_num 1))
    return S  # 返回生成的状态


def aggreg_state(state, Z_cluster_num, X, M_servers, I_citys, L_levels):
    # 函数2
    # 定义一个函数，用于根据给定的参数将复杂的状态 S 压缩成一个简化的状态 barS。
    # 输入:
    #   S: 当前状态，一个复杂的结构，包含两部分信息：
    #      - 一个数组，表示每个城市每个等级的数量(n_il)。
    #      - 一个列表，表示服务员和他们服务的城市及工作日(i_m, w_m)
    #   Z_cluster_num: 一个整数，表示将城市分成多少个聚类。
    #   X: 一个整数，用于计算 N 矩阵中的元素值。
    #   M_servers: 服务员的总数。
    #   I_citys: 城市的总数。
    #   L_levels: 等级的总数。
    # 输出:
    #   barS: 一个元组，表示压缩后的状态，包含以下三个部分：
    #         - N: 一个二维数组，表示每个聚类的等级之和。
    #         - g: 一个数组，表示每个聚类的状态。
    #         - w: 一个整数，表示第一个服务员的工作日数。

    # 计算正在工作的服务员的数量, S[1][m_server][1]即S1_servers[m_server][1]即w_m
    barM = np.sum([1 for m_server in range(M_servers) if state[1][m_server][1] != 0]) # 距离放假时间不等于0
    # 根据城市数量和设定的簇数，将城市分成Z个簇
    cluster = split_list(I_citys, Z_cluster_num)
    # 计算实际的簇数，考虑到可能会有余数
    # num_cluster = divide_reminder(I_citys, Z_cluster_num) # 这个可以替换为下面的
    num_cluster = np.ceil(I_citys / Z_cluster_num).astype(int) # 向上取整
    # 初始化表示各簇状态的数组g
    g = np.zeros(num_cluster)
    
    # 压缩状态的第二部分：计算每个簇的状态
    for z_cluster in range(num_cluster):
        # 统计每个簇中有多少业务员正在工作 
        e_z = np.sum([1 for m_server in range(M_servers) if state[1][m_server][0] in cluster[z_cluster]])
        # 根据工作的业务员数量设置簇的状态
        if e_z == 0:
            g[z_cluster] = 0  # 无业务员工作
        elif e_z <= barM / num_cluster:
            g[z_cluster] = 1  # 工作业务员数量低于或等于平均值
        else:
            g[z_cluster] = 2  # 工作业务员数量高于平均值
    
    # 获取第一个业务员的工作量
    w = state[1][0][1]
    # 压缩状态的第一部分：计算每个簇中各等级的数量总和
    N = np.zeros((num_cluster, L_levels))  # 初始化N矩阵
    
    i_all = [i for i in cluster[z_cluster]]
    # for z_cluster in range(num_cluster):
    #     for l in range(L_levels):
    #         # 对每个簇的每个等级，计算其数量总和，但不超过X
            
    #         N[z_cluster][l] = min(X, 
    #                               np.sum([state[0][i-1][l] for i in cluster[z_cluster]]))
    for z_cluster in range(num_cluster):
        for l in range(L_levels):
            # 对每个簇的每个等级，计算其数量总和，但不超过X
            task_sum = sum(float(state[0][i-1][l]) for i in cluster[z_cluster])
            N[z_cluster][l] = min(X, task_sum)
    # 将计算出的N矩阵、簇的状态数组g和第一个业务员的工作量w组合成新的压缩状态barS
    barS = (N, g, w)        
    return barS

# 将城市列表平均分成Z个聚类
def split_list(I_citys, Z_cluster_num)->list:
    # 创建一个从1到I_citys的城市索引列表
    arr_city_idx = list(range(1, I_citys + 1))
    # 调用函数处理实际的分割
    return split_array_given_array(arr_city_idx, Z_cluster_num)



def split_array_given_array(arr_city_idx, Z_cluster_num)->list:
    """
    将输入数组分割成长度为 Z_cluster_num 的子数组列表。如果数组不能被 Z_cluster_num 整除，
    那么最后一个子数组将包含所有剩余的元素。
    
    参数:
    arr_city_idx (list): 需要被分割的输入数组。
    Z_cluster_num (int): 每个子数组的期望长度。
    
    返回值:
    list: 长度为 Z_cluster_num 的子数组列表，除了可能的最后一个子数组，它包含所有剩余的元素。
    """
    result = []  # 结果列表，用来存储所有的子数组
    quotient = len(arr_city_idx) // Z_cluster_num  # 计算整除的商，即完整子数组的数量
    remainder = len(arr_city_idx) % Z_cluster_num  # 计算余数，即最后一个子数组的元素数量
    
    # 划分可以整除的数组部分
    for i in range(quotient):
        sub_array = arr_city_idx[i * Z_cluster_num:(i + 1) * Z_cluster_num]  # 获取从 i*Z_cluster_num 到 (i+1)*Z_cluster_num 的子数组
        result.append(sub_array)  # 将子数组添加到结果列表中
    
    # 如果有余数，则处理剩余部分
    if remainder > 0:
        sub_array = arr_city_idx[-remainder:]  # 获取数组最后余数个元素形成的子数组
        result.append(sub_array)  # 将子数组添加到结果列表中
    
    return result  # 返回结果列表

def divide_reminder(num, divisor):
    """
    将一个整数除以另一个整数，并将结果向上取整。
    
    参数:
    num (int): 被除数。
    divisor (int): 除数。
    
    返回值:
    int: 向上取整后的商。
    """
    quotient = num // divisor  # 计算整除的商
    remainder = num % divisor  # 计算余数
    
    # 如果存在余数，则将商向上取整
    if remainder > 0:
        quotient += 1  # 余数大于0，商加一
    
    return quotient  # 返回向上取整后的商

def func4(S, mathscr_L):
    # 假设 len(mathscr_L) = 7
    # 假设 S 和 mathscr_L 的数据结构如所描述
    # S 是一个二元组 (S0_tasks, S1_servers)，其中 S0_tasks 是任务矩阵
    # mathscr_L 是业务员的等级集合，例如 [l_1, l_2, ..., l_M]
    #S0 (I_citys x_max_task_num L_levels)
    # 首先计算 N_1 和 N_2
    L_levels = 5
    N_1 = [sum(1 for l_m, (i_m, w_m) in zip(mathscr_L, S[1]) if l_m == j and w_m != 0) for j in range(1, L_levels+1)]
    N_2 = [sum(S[0][i][j] for i in range(len(S[0]))) for j in range(L_levels)]
    # 初始化分类后的等级列表
    mathcal_L = []
    current_class = []
    total_N_1 = 0
    total_N_2 = 0

    for j in range(1, L_levels+1):
        total_N_1 += N_1[j-1]
        total_N_2 += N_2[j-1]
        current_class.append(j)

        if total_N_1 <= total_N_2:
            # 当 N_1 总和小于等于 N_2 总和时，终止当前类的添加
            mathcal_L.append(tuple(current_class))
            current_class = []
            total_N_1 = 0
            total_N_2 = 0

    if current_class:
        # 添加最后一个类
        mathcal_L.append(tuple(current_class))

    return mathcal_L, N_1, N_2

def func3(S, L_mathscr, H_home_of_server, r1_reward_for_level, c1_city_cost, c2):
    """
    生成状态 S 到决策 A 的函数，通过解决线性规划问题来最大化收益 R_total_reward(S, A)。

    参数:
    S (tuple): 当前状态，包含任务矩阵和服务员信息。
    L_mathscr (list): 服务员的等级列表。
    H_home_of_server (list): 服务员的家位置列表。
    r1_reward_for_level (list): 每个等级的收益列表。
    c1_city_cost (list of list): I×I 的成本矩阵。
    c2 (float): 常数成本。

    返回:
    list: 最优决策 A，包含每个服务员的位置和等级。
    """
    n_il, servers_info = S
    
    # n_il, servers_info = S
    # M_servers = len(servers_info)  # 服务员数量
    # I_citys = len(n_il)          # 城市数量
    # # L_max = len(r1_reward_for_level) - 1    # 最大等级

    M_servers = len(servers_info)  # 服务员数量
    I_citys = len(n_il)          # 城市数量
    L_max = max(L_mathscr) # 最大等级
    # print(f"{M_servers=} {I_citys=} {L_max=} {L_mathscr=}")

    # 创建问题实例
    prob = pulp.LpProblem("Optimal_Server_Assignment", pulp.LpMaximize)

    # 定义决策变量 y_{mil} 为二元变量
    y = pulp.LpVariable.dicts("y", 
                              ((m, i, l) for m in range(M_servers) for i in range(I_citys) for l in range(L_max+1)), 
                              cat=pulp.LpBinary)

    # 40 
    # I_citys=26 
    # L_max=5
    # 目标函数
    # 假设 L_max 和 r1_reward_for_level 已经定义

    prob += pulp.lpSum(
        r1_reward_for_level[l1] * y[m1, i1, l1] - c1_city_cost[servers_info[m1][0]-1][i1] * y[m1, i1, l1]
                       for m1 in range(M_servers) for i1 in range(I_citys)
                       for l1 in range(0, L_max+1))\
                        - c2 * pulp.lpSum(n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in range(M_servers)) for i in range(I_citys)
                       for l in range(1, L_max+1))

    # 添加约束
    # 每个服务员只能分配到一个地点和等级
    for m in range(M_servers):
        prob += pulp.lpSum(y[m, i, l] for i in range(I_citys) for l in range(L_max+1)) == 1

    # 服务员不工作时，分配到家乡的等级0
    for m, (im, wm) in enumerate(servers_info):
        if wm == 0:
            prob += y[m, H_home_of_server[m], 0] == 1

    # 服务员工作时，必须分配到合适的等级和城市
    for m, (im, wm) in enumerate(servers_info):
        if wm > 0:
            prob += pulp.lpSum(y[m, i, l] for i in range(I_citys) for l in range(L_mathscr[m], L_max+1) if n_il[i][l-1] > 0) == 1

    # 资源使用不超过可用数量
    for i in range(I_citys):
        for l in range(1, L_max+1):
            prob += pulp.lpSum(y[m, i, l] for m in range(M_servers)) <= n_il[i][l-1]

    # 求解问题
    prob.solve()

    # 解析结果
    result = [(m, i+1, l) for m in range(M_servers) for i in range(I_citys) for l in range(L_max+1) if pulp.value(y[m, i, l]) == 1]
    return result


def func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1_reward_for_level, c1_city_cost, c2):
    """
    生成状态 S 到决策 Y_best_allocation 的函数,通过解决线性规划问题来最大化收益 R_total_reward(S, Y_best_allocation)。

    参数:
    S (tuple): 当前状态,包含任务矩阵和服务员信息。
    mathcal_L (list): 分类后的等级列表。
    mathscr_L (list): 所有服务员的等级列表
    N_1 (list): 每个等级的服务员数量。
    N_2 (list): 每个等级的任务数量。
    H_home_of_server (list): 服务员的家位置列表。
    r1_reward_for_level (list): 每个等级的收益列表。
    c1_city_cost (list of list): I×I 的成本矩阵。
    c2 (float): 常数成本。

    返回:
    list: 最优决策 Y_best_allocation,包含每个服务员的位置和等级。
    R_total_reward, 总收益
    """
    n_il, servers_info = S
    M_servers = len(servers_info)  # 服务员数量
    I_citys = len(n_il)          # 城市数量
    L_max = [max(l) for l in mathcal_L]  # 最大等级
    # 步骤1:安排放假的员工回家
    C_h = sum(c1_city_cost[servers_info[m][0]-1][H_home_of_server[m]] for m in range(M_servers) if servers_info[m][1] == 0)
    
    R_total_reward = -C_h  # 初始化总收益为负的回家成本

    Y_best_allocation = [None] * M_servers  # 初始化最优决策 Y_best_allocation
    
    Y_set = []
    # 步骤2:对每个等级类独立进行员工分配
    for L_set, l_max_L in zip(mathcal_L, L_max):
        M_servers_L = [m for m in range(M_servers) if servers_info[m][1] > 0 and mathscr_L[m] in L_set]  # 该等级类下工作的员工集合
        I_citys_L = [i for i in range(I_citys) if any(n_il[i][l-1] > 0 for l in L_set)]  # 该等级类下有任务需求的城市集合
        
        # 创建问题实例
        prob = pulp.LpProblem(f"Optimal_Server_Assignment_Level_{L_set}", pulp.LpMaximize)

        # 定义决策变量 y_{mil} 为二元变量
        y = pulp.LpVariable.dicts("y", 
                                  ((m, i, l) for m in M_servers_L for i in I_citys_L for l in L_set), 
                                  cat=pulp.LpBinary)

        # 定义一个辅助变量,存储服务员分配的收益和城市成本
        # 这个表达式计算了每个服务员分配到每个城市每个等级任务的收益和成本

        reward_cost_expr = pulp.lpSum(
            r1_reward_for_level[l1-1] * y[m1, i1, l1] - c1_city_cost[servers_info[m1][0]-1][i1] * y[m1, i1, l1] 
            for m1 in M_servers_L for i1 in I_citys_L for l1 in L_set
        )

        # 目标函数
        # 根据等级类型选择
        # 如果每个等级的服务员数量小于等于任务数量,则最大化收益减去未分配任务的惩罚
        if sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set):
            prob += reward_cost_expr - c2 * pulp.lpSum(
                n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in M_servers_L)
                for i in I_citys_L for l in L_set
            )
        # 否则,只最大化收益
        else:
            prob += reward_cost_expr

        # 添加约束
        for m in M_servers_L: 
            # 每个工作中的服务员 m,要求其被分配到城市 i 提供的服务等级 l 必须不低于他自身的服务等级 L_mathscr[m]
            # 且只能被分配到一个城市提供一种等级的服务。
            prob += pulp.lpSum(y[m, i, l] for i in I_citys_L for l in L_set if l >= mathscr_L[m]) == 1

        # 遍历每个城市和等级
        for i in I_citys_L:
            for l in L_set:
                # 如果每个等级的服务员数量小于等于任务数量,则添加约束:
                # 每个城市每个等级分配的服务员数量小于等于该等级任务数量
                if sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set):
                    prob += pulp.lpSum(y[m, i, l] for m in M_servers_L) <= n_il[i][l-1]
                # 否则,添加约束:
                # 每个城市每个等级分配的服务员数量等于该等级任务数量
                else:
                    prob += pulp.lpSum(y[m, i, l] for m in M_servers_L) == n_il[i][l-1]

        # 求解问题
        prob.solve()

        
        # 遍历每个服务员、城市和等级
        for m in M_servers_L:
            for i in I_citys_L:
                for l in L_set:
                    # 初始化一个临时列表,用于存储分配结果
                    Y_sub_set = []
                    # 如果当前服务员被分配到某个城市某个等级
                    if pulp.value(y[m, i, l]) == 1:
                        # 将当前服务员、城市(城市编号从1开始)和等级的分配结果存储到Y_best_allocation字典中
                        Y_best_allocation[m] = (m, i+1, l)
                        # 中断内层循环,开始处理下一个服务员
                        break

        # 更新总收益
        R_total_reward += pulp.value(prob.objective)

        # 提取结果
        # Y_L = [(i, l) for m in M_servers_L for i in I_citys_L for l in L_set if y[m, i, l].value() == 1]
        # Y_set.append(Y_L)

    
    # 步骤3:安排放假的员工
    for m in range(M_servers):
        if servers_info[m][1] == 0:
            Y_best_allocation[m] = (m, H_home_of_server[m]+1, 0)  # 城市编号从1开始
    # # 步骤3:计算总收益
    # R_total_reward = sum(prob.objective.value() for L in L_set) - C_h
    #     Y_set = []
        # 解析结果


    return Y_best_allocation, R_total_reward


def func6(S, mathcal_L, mathscr_L, N_1, N_2):
    """
    生成状态 S 的决策空间 A,满足约束条件。

    参数:
    S (tuple): 当前状态,包含任务矩阵和服务员信息。
    mathcal_L (list): 分类后的等级列表。
    mathscr_L (list): 所有服务员的等级列表
    N_1 (list): 每个等级的服务员数量。
    N_2 (list): 每个等级的任务数量。

    返回:
    list: 决策空间 A,包含每个服务员的所有可能决策。
    """
    n_il, servers_info = S
    M_servers = len(servers_info)  # 服务员数量
    I_citys = len(n_il)          # 城市数量

    A = [[] for _ in range(M_servers)]  # 初始化决策空间 A
    A_mil =  A

    # 对每个服务员生成可能的决策
    for m in range(M_servers):
        if servers_info[m][1] == 0:  # 服务员 m 放假
            A[m].append((servers_info[m][0], 0))  # 放假的服务员只有一个决策,即回家
        else:  # 服务员 m 工作
            for L_set in mathcal_L:  # 遍历每个等级类
                if mathscr_L[m] in L_set:  # 如果服务员 m 的等级属于当前等级类
                    for i in range(I_citys):  # 遍历每个城市
                        for l in L_set:  # 遍历当前等级类的每个等级
                            if l >= mathscr_L[m]:  # 如果当前等级不低于服务员 m 的等级
                                if (sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set) and 
                                    sum(1 for a in A[m] if a[0] == i+1) < sum(n_il[i][l-1] for l in L_set)) or \
                                   (sum(N_1[l-1] for l in L_set) > sum(N_2[l-1] for l in L_set) and
                                    sum(1 for a in A[m] if a[0] == i+1 and a[1] == l) < n_il[i][l-1]):
                                    # 如果满足约束条件,则将决策添加到服务员 m 的决策空间中
                                    A[m].append((i+1, l))  # 城市编号从1开始

    return A

def func7(T, x_max_task_num, lambda_il):
    # 生成了每日新到达的任务?
    # T: 表示时间周期，例如天数
    # x_max_task_num: 矩阵元素的最大取值
    # lambda_il: 泊松分布的率参数矩阵 (I_citys x_max_task_num L_levels)

    # 获取 lambda_il 的维度为 I_citys 和 L_levels
    I_citys, L_levels = lambda_il.shape

    # 初始化三维数组
    arriving_taskss = np.zeros((T, I_citys, L_levels), dtype=int)
    
    # 生成每个时间步的 I_citys x_max_task_num L_levels 矩阵
    for t in range(T):
        for i in range(I_citys):
            for l in range(L_levels):
                # 使用泊松分布生成矩阵元素
                arriving_taskss[t, i, l] = min(np.random.poisson(lambda_il[i, l]), x_max_task_num)
    
    return arriving_taskss

func1 = generate_state
func2 = aggreg_state

func3 = func3
func4 = func4
func5 = func5
func6 = func6
func7 = func7

In [68]:
I_citys = 26
L_levels = 5
W_workdays = 6
M_servers = 40
x_max_task_num = 2
np.random.seed(42)
H_home_of_server = [1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,]
lambd = np.random.rand(I_citys, L_levels)
func1 = generate_state
state = func1(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd)
print(state)

H_home_of_server=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
S1_servers=[(26, 3), (5, 2), (25, 5), (14, 4), (5, 3), (6, 6), (14, 0), (20, 5), (12, 4), (26, 1), (7, 4), (1, 3), (5, 5), (4, 6), (10, 1), (5, 5), (2, 3), (10, 3), (19, 1), (1, 1), (13, 4), (16, 3), (3, 6), (17, 1), (20, 3), (18, 3), (1, 2), (19, 0), (5, 2), (3, 3), (1, 0), (11, 1), (13, 3), (14, 3), (19, 1), (3, 1), (7, 6), (26, 0), (26, 4), (26, 1)]
(array([[0, 0, 0, 0, 1],
       [0, 0, 0, 2, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 1, 1],
       [1, 0, 0, 2, 1],
       [0, 0, 0, 0, 2],
       [0, 0, 2, 1, 0],
       [0, 2, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 2, 0, 0],
       [0, 0, 0, 1, 3],
       [1, 0, 0, 0, 0],
       [1, 3, 0, 1, 2],
       [0, 0, 1, 0, 0],
       [1, 1, 2, 2, 1],
       [0, 0, 4, 0, 2],
       [2, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]

In [69]:
Z_cluster_num=3
X=3
M_servers=40
I_citys=26
L_levels=5
barS=aggreg_state(state, Z_cluster_num, X, M_servers, I_citys, L_levels)
S = state
# (位置i_m, 放假天数w_m)
rest_day_L = [server_restday[1] for server_restday in state[1]]

In [70]:
S = state
# (位置i_m, 放假天数w_m)
mathscr_L = [1,1,1,2,3,4,5,2,3,5, 1,1,1,2,3,4,5,2,3,5,1,1,1,2,3,4,5,2,3,5,1,1,1,2,3,4,5,2,3,5,]
mathcal_L, N_1, N_2 = func4(S, mathscr_L)
print(mathcal_L, N_1, N_2)

[(1,), (2,), (3,), (4,), (5,)] [11, 6, 8, 4, 7] [14, 8, 16, 15, 16]


In [71]:
S = state
n_il, servers_info = S
L_mathscr = mathscr_L
H_home_of_server = H_home_of_server
r1_reward_for_level  = [0, 3500, 3000, 2500, 2000, 1500]
# 假设 I_citys 是城市的数量，这个值应该根据你的具体情况来设置
I_citys = len(n_il)  # 以 n_il 变量中元素的数量来确定城市数量

# 生成随机的成本矩阵 c1_city_cost
random.seed(42)
c1_city_cost = [[0 if i == j else random.randint(100, 500) for j in range(I_citys)] for i in range(I_citys)]
c2 = 100
A =func3(S, L_mathscr, H_home_of_server, r1_reward_for_level, c1_city_cost, c2)

In [72]:
A

[(0, 26, 1),
 (1, 15, 1),
 (2, 4, 1),
 (3, 11, 2),
 (4, 13, 3),
 (5, 23, 4),
 (6, 8, 0),
 (7, 11, 2),
 (8, 10, 3),
 (9, 14, 5),
 (10, 7, 1),
 (11, 18, 1),
 (12, 5, 1),
 (13, 6, 2),
 (14, 10, 3),
 (15, 8, 4),
 (16, 2, 5),
 (17, 16, 2),
 (18, 19, 3),
 (19, 19, 5),
 (20, 20, 1),
 (21, 16, 1),
 (22, 6, 1),
 (23, 18, 2),
 (24, 22, 3),
 (25, 18, 4),
 (26, 1, 5),
 (27, 9, 0),
 (28, 13, 3),
 (29, 19, 5),
 (30, 2, 0),
 (31, 8, 1),
 (32, 20, 1),
 (33, 16, 2),
 (34, 19, 3),
 (35, 10, 4),
 (36, 7, 5),
 (37, 9, 0),
 (38, 26, 3),
 (39, 14, 5)]

In [73]:
Y_best_allocation, R_total_reward = func5(S,mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1_reward_for_level, c1_city_cost, c2)

In [74]:
Y_best_allocation

[(0, 26, 1),
 (1, 5, 1),
 (2, 4, 1),
 (3, 16, 2),
 (4, 13, 3),
 (5, 23, 4),
 (6, 8, 0),
 (7, 11, 2),
 (8, 10, 3),
 (9, 14, 5),
 (10, 7, 1),
 (11, 18, 1),
 (12, 15, 1),
 (13, 6, 2),
 (14, 10, 3),
 (15, 8, 4),
 (16, 2, 5),
 (17, 16, 2),
 (18, 19, 3),
 (19, 19, 5),
 (20, 20, 1),
 (21, 16, 1),
 (22, 6, 1),
 (23, 18, 2),
 (24, 22, 3),
 (25, 18, 4),
 (26, 1, 5),
 (27, 9, 0),
 (28, 13, 3),
 (29, 19, 5),
 (30, 2, 0),
 (31, 8, 1),
 (32, 20, 1),
 (33, 11, 2),
 (34, 19, 3),
 (35, 10, 4),
 (36, 7, 5),
 (37, 9, 0),
 (38, 26, 3),
 (39, 14, 5)]

In [75]:
A_all = func6(S, mathcal_L, mathscr_L, N_1, N_2)
len(A_all)

40

In [76]:
A_all_lens = [len(A) for A in A_all]
print(len(A_all), A_all_lens)

40 [13, 13, 13, 5, 9, 10, 1, 5, 9, 10, 13, 13, 13, 5, 9, 10, 10, 5, 9, 10, 13, 13, 13, 5, 9, 10, 10, 1, 9, 10, 1, 13, 13, 5, 9, 10, 10, 1, 9, 10]


In [77]:
# 示例参数
T = 7  # 时间步数量
x_max_task_num = 3  # 最大值
I_citys = 26  # 城市数量
L_levels = 5  # 等级数量

np.random.seed(42)
lambda_il = np.random.rand(I_citys, L_levels)  # 生成率参数矩阵

# 生成arriving_tasks_i
arriving_tasks_for_T = func7(T, x_max_task_num, lambda_il)
print(arriving_tasks_for_T.shape, arriving_tasks_for_T)

(7, 26, 5) [[[0 0 0 0 1]
  [0 0 0 2 1]
  [0 0 1 0 0]
  [1 0 0 1 0]
  [1 0 0 0 0]
  [1 1 0 0 0]
  [1 0 0 1 1]
  [1 0 0 2 1]
  [0 0 0 0 2]
  [0 0 2 1 0]
  [0 2 0 1 0]
  [1 0 0 0 0]
  [0 0 2 0 0]
  [0 0 0 1 3]
  [1 0 0 0 0]
  [1 3 0 1 2]
  [0 0 1 0 0]
  [1 1 2 2 1]
  [0 0 3 0 2]
  [2 0 0 0 0]
  [0 1 0 0 0]
  [1 0 1 0 0]
  [0 0 1 3 0]
  [1 0 0 0 2]
  [0 0 0 0 0]
  [1 0 2 0 0]]

 [[0 0 1 0 0]
  [0 0 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [2 0 0 0 0]
  [2 0 0 0 0]
  [0 2 0 0 0]
  [0 0 0 0 1]
  [0 0 0 2 0]
  [1 0 3 0 0]
  [0 2 2 2 0]
  [1 0 1 0 2]
  [0 0 0 0 0]
  [2 0 1 0 2]
  [0 0 0 1 0]
  [1 2 0 1 0]
  [3 0 1 0 1]
  [0 0 0 0 0]
  [0 1 2 0 1]
  [1 1 1 1 0]
  [1 1 0 1 1]
  [0 2 1 0 0]
  [0 0 0 3 0]
  [2 3 0 2 0]
  [0 2 0 0 2]
  [0 0 0 0 3]]

 [[2 0 0 0 1]
  [0 0 1 0 1]
  [0 2 2 0 0]
  [0 0 1 0 0]
  [0 0 0 0 1]
  [1 0 1 1 0]
  [0 0 0 2 1]
  [0 0 0 0 0]
  [0 0 0 2 0]
  [0 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 2]
  [0 1 0 0 0]
  [0 0 0 0 1]
  [0 0 0 1 0]
  [0 1 0 0 0]
  [0 0 0 0 1]
  [0 1 0 1 1]
  [0 

In [78]:

    # def func5_with_V(self, t, S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1_reward_for_level, c1_city_cost, c2, V, Xi_arriving_tasks):
    #     """
    #     生成状态 S 到决策 Y_best_allocation 的函数,通过解决线性规划问题来最大化收益 R_total_reward(S, Y_best_allocation) + V(t+1, S').

    #     参数:
    #     t (int): 当前时间步。
    #     S (tuple): 当前状态,包含任务矩阵和服务员信息。
    #     mathcal_L (list): 分类后的等级列表。
    #     mathscr_L (list): 所有服务员的等级列表
    #     N_1 (list): 每个等级的服务员数量。
    #     N_2 (list): 每个等级的任务数量。
    #     H_home_of_server (list): 服务员的家位置列表。
    #     r1_reward_for_level (list): 每个等级的收益列表。
    #     c1_city_cost (list of list): I×I 的成本矩阵。
    #     c2 (float): 常数成本。
    #     V (ValueFunction): 值函数对象。

    #     返回:
    #     list: 最优决策 Y_best_allocation,包含每个服务员的位置和等级。
    #     R_total_reward, 总收益
    #     """
    #     n_il, servers_info = S
    #     M_servers = len(servers_info)  # 服务员数量
    #     I_citys = len(n_il)          # 城市数量
    #     L_max = [max(l) for l in mathcal_L]  # 最大等级

    #     # 步骤1:安排放假的员工回家
    #     C_h = sum(c1_city_cost[servers_info[m][0]-1][H_home_of_server[m]] for m in range(M_servers) if servers_info[m][1] == 0)
        
    #     R_total_reward = -C_h  # 初始化总收益为负的回家成本

    #     Y_best_allocation = [None] * M_servers  # 初始化最优决策 Y_best_allocation
        
    #     # 步骤2:对每个等级类独立进行员工分配
    #     for L_set, l_max_L in zip(mathcal_L, L_max):
    #         M_servers_L = [m for m in range(M_servers) if servers_info[m][1] > 0 and mathscr_L[m] in L_set]  # 该等级类下工作的员工集合 工作：(servers_info[m][1] > 0) 该等级类for m in range(M_servers) mathscr_L[m] in L_set 
    #         I_citys_L = [i for i in range(I_citys) if any(n_il[i][l-1] > 0 for l in L_set)]  # 该等级类下有任务需求的城市集合

    #         # 创建问题实例
    #         prob = pulp.LpProblem(f"Optimal_Server_Assignment_Level_{L_set}", pulp.LpMaximize)

    #         # 定义决策变量 y_{mil} 为二元变量
    #         y = pulp.LpVariable.dicts("y", 
    #                                 ((m, i, l) for m in M_servers_L for i in I_citys_L for l in L_set), 
    #                                 cat=pulp.LpBinary)
    #         # 定义一个辅助函数,计算下一个状态的价值函数
    #         def calculate_next_state_value(arriving_tasks):
    #             """
    #             计算下一个状态的价值函数
    #             Args:
    #                 arriving_tasks (list): 当前时间步到达的任务
    #             Returns:
    #                 float: 下一个状态的价值函数
    #             """
    #             # 计算下一个状态
    #             new_state = tuple(n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in M_servers_L) + arriving_tasks[i][l-1] 
    #                             for i in range(I_citys) for l in L_set)
    #             # 压缩状态
    #             S_bar_next = func2(new_state, self.Z_cluster_num, self.X, M_servers, I_citys, self.L_levels)
    #             # 更新价值函数的计数
    #             count = V.get_count(t+1, S_bar_next)
    #             # 返回下一个状态的价值函数
    #             return V.get(t+1, S_bar_next) * (1 - 1 / count)

    #         # 等一下这里不太对，这里是对每个集合内的
    #         # 目标函数
    #         # 定义一个辅助变量,存储服务员分配的收益和城市成本
    #         reward_cost_expr = pulp.lpSum(
    #             r1_reward_for_level[l1-1] * y[m1, i1, l1] - c1_city_cost[servers_info[m1][0]-1][i1] * y[m1, i1, l1] 
    #             for m1 in M_servers_L for i1 in I_citys_L for l1 in L_set
    #         )

    #         # 根据等级类型选择目标函数
    #         # 如果每个等级的服务员数量小于等于任务数量,则最大化收益减去未分配任务的惩罚,加上后续时间步的预期价值
    #         if sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set):
    #             prob += reward_cost_expr \
    #                 - c2 * pulp.lpSum(n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in M_servers_L) 
    #                                     for i in I_citys_L for l in L_set) \
    #                 + pulp.lpSum(calculate_next_state_value(arriving_tasks) 
    #                                 for arriving_tasks in Xi_arriving_tasks)
    #         # 否则,只最大化收益,加上后续时间步的预期价值
    #         else:
    #             prob += reward_cost_expr \
    #                 + pulp.lpSum(calculate_next_state_value(arriving_tasks) 
    #                                 for arriving_tasks in Xi_arriving_tasks)

    #         # 添加约束
    #         for m in M_servers_L: 
    #             # 每个工作中的服务员 m,要求其被分配到城市 i 提供的服务等级 l 必须不低于他自身的服务等级 L_mathscr[m]
    #             # 且只能被分配到一个城市提供一种等级的服务。
    #             prob += pulp.lpSum(y[m, i, l] for i in I_citys_L for l in L_set if l >= mathscr_L[m]) == 1

    #         # 添加约束条件
    #         for i in I_citys_L:
    #             for l in L_set:
    #                 # 如果每个等级的服务员数量小于等于任务数量,则添加约束:
    #                 # 每个城市每个等级分配的服务员数量小于等于该等级任务数量
    #                 if sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set):
    #                     prob += pulp.lpSum(y[m, i, l] for m in M_servers_L) <= n_il[i][l-1]
    #                 # 否则,添加约束:
    #                 # 每个城市每个等级分配的服务员数量等于该等级任务数量
    #                 else:
    #                     prob += pulp.lpSum(y[m, i, l] for m in M_servers_L) == n_il[i][l-1]

    #         # 求解问题
    #         prob.solve()

    #         # 记录最优分配结果
    #         for m in M_servers_L:
    #             for i in I_citys_L:
    #                 for l in L_set:
    #                     # 如果当前服务员被分配到某个城市某个等级
    #                     if pulp.value(y[m, i, l]) == 1:
    #                         # 将当前服务员、城市(城市编号从1开始)和等级的分配结果存储到Y_best_allocation字典中
    #                         Y_best_allocation[m] = (m, i+1, l)
    #                         # 中断内层循环,开始处理下一个服务员
    #                         break

    #         # 计算后续时间步的预期价值
    #         for arriving_tasks in Xi_arriving_tasks:
    #             # 计算下一个状态的价值函数
    #             R_total_reward += calculate_next_state_value(arriving_tasks)

    #         # 步骤3:安排放假的员工
    #         for m in range(M_servers):
    #             if servers_info[m][1] == 0:
    #                 Y_best_allocation[m] = (m, H_home_of_server[m]+1, 0)  # 城市编号从1开始

    #         return Y_best_allocation, R_total_reward

In [91]:
import numpy as np
import copy
import itertools

import os
import pickle

class ValueFunction:
    def __init__(self, T=7, file_path="./value_function.pkl"):
        self.values = {key: {} for key in range(T + 1)}  # 用字典存储状态-价值对
        self.count = {key: {} for key in range(T + 1)}  # 用字典存储状态出现次数
        self.file_path = file_path

    def set(self, t, state, value):
        """
        存储阶段t和状态state的价值value
        """
        if t not in self.values:
            self.values[t] = {}
            self.count[t] = {}
        if type(state) != str:
            
            state_str = process_state(state)
        else:
            state_str = state
        self.values[t][state_str] = value
        self.count[t][state_str] = self.count[t].get(state_str, 0) + 1

    def get(self, t, state):
        """
        获取阶段t和状态state的价值,如果状态不存在,则返回默认值0
        """
        state_str = process_state(state)
        if t not in self.values or state_str not in self.values[t]:
            self.values[t] = {}
            self.values[t][state_str] = 0
            return 0.0
        return self.values[t][state_str]

    def get_count(self, t, state):
        """
        获取阶段t和状态state的出现次数,如果状态不存在,则返回默认值1 表示当前出现一次
        """
        state_str = process_state(state)
        if t not in self.count or state_str not in self.count[t]:
            self.set(t, state_str, 0)
        return self.count[t][state_str]

    def save(self, file_path=None):
        """
        保存 ValueFunction 实例到文件
        """
        if file_path is None:
            file_path = self.file_path
        if file_path is None:
            raise ValueError("No file path provided for saving the ValueFunction object.")

        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, "wb") as f:
            pickle.dump(self, f)

    @classmethod
    def load(cls, file_path):
        """
        从文件中加载 ValueFunction 实例
        """
        with open(file_path, "rb") as f:
            return pickle.load(f)


def process_state(state):
    """
    将状态转换为NumPy数组，并返回转换后的状态。
    """
    state_str = str(state)
    return state_str

class Func8:
    
    pi = None  # 初始化策略
    I_citys = 26
    L_levels = 5
    W_workdays = 6
    M_servers = 40
    x_max_task_num = 2
    random.seed(42)
    lambd = np.random.rand(I_citys, L_levels)
    r1_reward_for_level  = [0, 3500, 3000, 2500, 2000, 1500]
    c2 = 100

    max_iter = 10000
    T = 7

    Z_cluster_num=3
    X=3
    def __init__(self):
        
        self.V = ValueFunction()  # 初始化值函数
        self.H_home_of_server = [random.randint(1, self.I_citys) for _ in range(self.M_servers)] # 随机家的位置
        self.c1_city_cost = [[0 if i == j else random.randint(100, 500) 
                              for j in range(self.I_citys)] for i in range(self.I_citys)]
        self.lambda_il = np.random.rand(self.I_citys, self.L_levels)  # 生成率参数矩阵
        self.arriving_tasks_for_T = func7(self.T, self.x_max_task_num, self.lambda_il)
        self.mathscr_L = [random.randint(1, self.L_levels) for _ in range(self.M_servers)]
         
        # mathcal_L, N_1, N_2 = func4(S=S, mathscr_L=self.mathscr_L)
        # self.mathcal_L = mathcal_L
        # self.N_1 = N_1
        # self.N_2 = N_2

    def func3_with_V(self, S, t, L_mathscr, H_home_of_server, r1_reward_for_level, c1_city_cost, c2, V, Xi_arriving_tasks):
        """
        生成状态 S 到决策 A 的函数,通过解决线性规划问题来最大化收益 R_total_reward(S, A) + V(t+1, S')。

        参数:
        S (tuple): 当前状态,包含任务矩阵和服务员信息。
        L_mathscr (list): 服务员的等级列表。
        H_home_of_server (list): 服务员的家位置列表。
        r1_reward_for_level (list): 每个等级的收益列表。
        c1_city_cost (list of list): I×I 的成本矩阵。
        c2 (float): 常数成本。
        V (ValueFunction): 值函数对象。
        Xi_arriving_tasks (list of list): 在当前时间步到达的任务矩阵。

        返回:
        list: 最优决策 A,包含每个服务员的位置和等级。
        R_total_reward, 总收益
        """
        n_il, servers_info = S
        M_servers = len(servers_info)  # 服务员数量
        I_citys = len(n_il)          # 城市数量
        L_max = len(r1_reward_for_level) - 1    # 最大等级
        print(f"{M_servers=} {I_citys=} {L_max=}")
        # 创建问题实例
        prob = pulp.LpProblem("Optimal_Server_Assignment", pulp.LpMaximize)

        # 定义决策变量 y_{mil} 为二元变量
        y = pulp.LpVariable.dicts("y", 
                                ((m, i, l) for m in range(M_servers) for i in range(I_citys) for l in range(L_max+1)), 
                                cat=pulp.LpBinary)

        # 定义一个辅助函数,计算下一个状态的价值函数
        def calculate_next_state_value(arriving_tasks):
            """
            计算下一个状态的价值函数
            Args:
                arriving_tasks (list): 当前时间步到达的任务
            Returns:
                float: 下一个状态的价值函数
            """
            # new_servers_info = self.update_servers_info(servers_info, y, M_servers, I_citys, L_max)
            # 计算下一个状态, 减去根据决策变量 y 分配到下一个等级 l+1 的服务员数量 pulp.lpSum(y[m, i, l+1] for m in range(M_servers))
            new_state = [
                [[n_il[i][l] - pulp.lpSum(y[m, i, l] for m in range(M_servers)) \
                  + arriving_tasks[i][l] for l in range(L_max)] for i in range(I_citys)]
                ,[(i+1, servers_info[m][1]-1) if pulp.value(y[m, i, l]) == 1 else (servers_info[m][0], servers_info[m][1])
      for m in range(M_servers) for i in range(I_citys) for l in range(L_max+1)]
            ]
            # 压缩状态
            S_bar_next = self.aggreg_state(new_state, self.Z_cluster_num, self.X, M_servers, I_citys, self.L_levels)
            # 更新价值函数的计数
            count = V.get_count(t+1, S_bar_next)
            # 返回下一个状态的价值函数
            return V.get(t+1, S_bar_next) * (1 - 1 / count)


        # prob += pulp.lpSum(
        # r1_reward_for_level[l1] * y[m1, i1, l1] - c1_city_cost[servers_info[m1][0]-1][i1] * y[m1, i1, l1]
        #                for m1 in range(M_servers) for i1 in range(I_citys)
        #                for l1 in range(0, L_max+1))\
        #                 - c2 * pulp.lpSum(n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in range(M_servers)) for i in range(I_citys)
        #                for l in range(1, L_max+1))


        # 目标函数
        
        prob += pulp.lpSum(
            r1_reward_for_level[l1] * y[m1, i1, l1] - c1_city_cost[servers_info[m1][0]-1][i1] * y[m1, i1, l1]
                        for m1 in range(M_servers) for i1 in range(I_citys)
                        for l1 in range(L_max+1)) \
                - c2 * pulp.lpSum(n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in range(M_servers)) for i in range(I_citys)
                        for l in range(1, L_max+1)) \
                +  calculate_next_state_value(Xi_arriving_tasks)
                # + pulp.lpSum(calculate_next_state_value(Xi_arriving_tasks)) / len(Xi_arriving_tasks)

        
        # 添加约束
        # 每个服务员只能分配到一个地点和等级
        for m in range(M_servers):
            prob += pulp.lpSum(y[m, i, l] for i in range(I_citys) for l in range(L_max+1)) == 1

        # 服务员不工作时，分配到家乡的等级0
        for m, (im, wm) in enumerate(servers_info):
            if wm == 0:
                prob += y[m, H_home_of_server[m]-1, 0] == 1

        # 服务员工作时，必须分配到合适的等级和城市
        for m, (im, wm) in enumerate(servers_info):
            if wm > 0:
                prob += pulp.lpSum(y[m, i, l] for i in range(I_citys) for l in range(L_mathscr[m], L_max+1) if n_il[i][l-1] > 0) == 1

        # 资源使用不超过可用数量
        for i in range(I_citys):
            for l in range(1, L_max+1):
                prob += pulp.lpSum(y[m, i, l] for m in range(M_servers)) <= n_il[i][l-1]


        # 求解问题
        prob.solve()

        # 解析结果
        result = [(m, i+1, l) for m in range(M_servers) for i in range(I_citys) for l in range(L_max+1) if pulp.value(y[m, i, l]) == 1]
        return result, pulp.value(prob.objective) # i[1,I_citys] l[0,L_max]

    
    def run(self):
        """
        带起始探索的强化学习算法,生成最优值函数和最优策略。

        返回:
        tuple: 包含最优状态、最优值函数和最优策略的元组。
        """
        # 初始化值函数和策略
        self.V = ValueFunction()
        self.pi = {}

        for i in range(self.max_iter):
            print(f"-------------迭代次数-{i=}--------------")
            # 生成初始状态
            S = func1(I_citys=self.I_citys,
                    L_levels=self.L_levels,
                    W_workdays=self.W_workdays,
                    M_servers=self.M_servers,
                    x_max_task_num=self.x_max_task_num,
                    H_home_of_server=self.H_home_of_server,
                    lambd=self.lambd)
            
            # 初始化轨迹
            trajectory = []

            for t in range(self.T):
                print(f"--------------{t=}--------------")
                Xi_arriving_tasks = self.arriving_tasks_for_T[t]

                # 获取最优决策和收益
                # S, t, L_mathscr, H_home_of_server, r1_reward_for_level, c1_city_cost, c2, V, Xi_arriving_tasks
                Y_best_allocation, R_total_reward = self.func3_with_V(S=S,t=t,
                                                                    L_mathscr=self.mathscr_L,
                                                                    H_home_of_server=self.H_home_of_server,
                                                                    r1_reward_for_level=self.r1_reward_for_level,
                                                                    c1_city_cost=self.c1_city_cost,
                                                                    c2=self.c2,
                                                                    V=self.V,
                                                                    Xi_arriving_tasks=Xi_arriving_tasks)

                # 记录轨迹
                trajectory.append((S, Y_best_allocation, R_total_reward))

                # 状态转移
                S_next = self.transition(S, Y_best_allocation, Xi_arriving_tasks)

                # 压缩状态
                S_bar_next = func2(S_next,
                                self.Z_cluster_num,
                                self.X,
                                self.M_servers,
                                self.I_citys,
                                self.L_levels)

                # 更新状态出现次数
                self.V.count[t][str(S_bar_next)] = 0
                self.V.count[t][str(S_bar_next)] += 1

                S = S_next

            # 逆向计算G和更新V
            G = [0] * (self.T + 1)
            for t in range(self.T - 1, -1, -1):
                S, Y_best_allocation, R_total_reward = trajectory[t]
                S_bar = func2(S, self.Z_cluster_num, self.X, self.M_servers, self.I_citys, self.L_levels)
                S_next = self.transition(S, Y_best_allocation, Xi_arriving_tasks)
                S_bar_next = func2(S_next, self.Z_cluster_num, self.X, self.M_servers, self.I_citys, self.L_levels)
                G[t] = R_total_reward + G[t + 1]
                count = self.V.get_count(t, S_bar)
                self.V.set(t, S_bar, self.V.get(t, S_bar) + (1 / count) * (G[t] - self.V.get(t, S_bar)))

            # 更新策略
            for t, (S, Y_best_allocation, _) in enumerate(trajectory):
                S_bar = func2(S, self.Z_cluster_num, self.X, self.M_servers, self.I_citys, self.L_levels)
                Xi_arriving_tasks = func7(t, self.x_max_task_num, self.lambda_il)
                Y_bar = Y_best_allocation
                S_bar_str = str(S_bar)
                if S_bar_str not in self.pi or self.V.get(t + 1, self.transition(S, Y_bar, Xi_arriving_tasks)) > self.V.get(t + 1, self.transition(S, self.pi[S_bar_str], Xi_arriving_tasks)):
                    self.pi[S_bar_str] = Y_bar

        # 找到最优状态
        S_opt = max(self.V.values[0], key=self.V.get)

        return S_opt, self.V, self.pi

    def transition(self, S, Y_best_allocation, Xi_arriving_tasks):
        # print("transiting")
        """
        状态转移函数。

        参数:
        S (tuple): 当前状态。
        Y_best_allocation (list): 当前决策。 # m, i[1,I_citys] l[0,L_max]
        Xi_arriving_tasks (np.array): 每个时间段新增任务的数量矩阵。

        返回:
        tuple: 下一个状态。
        """
        n_il, servers_info = S
        n_il_new = n_il + Xi_arriving_tasks - self.count_tasks(Y_best_allocation)
        servers_info_new = []
        for m in range(len(servers_info)):
            i, w = servers_info[m]
            if w > 0:
                servers_info_new.append((Y_best_allocation[m][1], w-1)) # 0 1 2  m i l
            else:
                servers_info_new.append((self.H_home_of_server[m], self.W_workdays))
        return (n_il_new, servers_info_new)

    def aggreg_state(self, state, Z_cluster_num, X, M_servers, I_citys, L_levels):
        # 函数2
        # 定义一个函数，用于根据给定的参数将复杂的状态 S 压缩成一个简化的状态 barS。
        # 输入:
        #   S: 当前状态，一个复杂的结构，包含两部分信息：
        #      - 一个数组，表示每个城市每个等级的数量(n_il)。
        #      - 一个列表，表示服务员和他们服务的城市及工作日(i_m, w_m)
        #   Z_cluster_num: 一个整数，表示将城市分成多少个聚类。
        #   X: 一个整数，用于计算 N 矩阵中的元素值。
        #   M_servers: 服务员的总数。
        #   I_citys: 城市的总数。
        #   L_levels: 等级的总数。
        # 输出:
        #   barS: 一个元组，表示压缩后的状态，包含以下三个部分：
        #         - N: 一个二维数组，表示每个聚类的等级之和。
        #         - g: 一个数组，表示每个聚类的状态。
        #         - w: 一个整数，表示第一个服务员的工作日数。

        # 计算正在工作的服务员的数量, S[1][m_server][1]即S1_servers[m_server][1]即w_m
        barM = np.sum([1 for m_server in range(M_servers) if state[1][m_server][1] != 0]) # 距离放假时间不等于0
        # 根据城市数量和设定的簇数，将城市分成Z个簇
        cluster = split_list(I_citys, Z_cluster_num)
        # 计算实际的簇数，考虑到可能会有余数
        # num_cluster = divide_reminder(I_citys, Z_cluster_num) # 这个可以替换为下面的
        num_cluster = np.ceil(I_citys / Z_cluster_num).astype(int) # 向上取整
        # 初始化表示各簇状态的数组g
        g = np.zeros(num_cluster)
        
        # 压缩状态的第二部分：计算每个簇的状态
        for z_cluster in range(num_cluster):
            # 统计每个簇中有多少业务员正在工作 
            e_z = np.sum([1 for m_server in range(M_servers) if state[1][m_server][0] in cluster[z_cluster]])
            # 根据工作的业务员数量设置簇的状态
            if e_z == 0:
                g[z_cluster] = 0  # 无业务员工作
            elif e_z <= barM / num_cluster:
                g[z_cluster] = 1  # 工作业务员数量低于或等于平均值
            else:
                g[z_cluster] = 2  # 工作业务员数量高于平均值
        
        # 获取第一个业务员的工作量
        w = state[1][0][1]
        # 压缩状态的第一部分：计算每个簇中各等级的数量总和
        N = np.zeros((num_cluster, L_levels))  # 初始化N矩阵
        
        i_all = [i for i in cluster[z_cluster]]

        for z_cluster in range(num_cluster):
            for l in range(L_levels):
                # 对每个簇的每个等级，计算其数量总和，但不超过X
                state_sum = sum(state[0][i-1][l] for i in cluster[z_cluster])
                # print(f"{state_sum=}")
                task_sum = pulp.value(state_sum)
                if task_sum is None:
                    task_sum = 0
                else:
                    print(f"{task_sum=} is not None")
                N[z_cluster][l] = min(X, task_sum)
                # print(f"{N[z_cluster][l]=}")
        # 将 N 和 g 转换为元组
        N = tuple(map(tuple, N))
        g = tuple(g)
        # 将计算出的N矩阵、簇的状态数组g和第一个业务员的工作量w组合成新的压缩状态barS
        barS = (N, g, w)        
        return barS
    
    # def count_tasks(self, Y_best_allocation):
    #     """
    #     统计决策Y中完成的任务数量。

    #     参数:
    #     Y_best_allocation (list): 决策。

    #     返回:
    #     np.array: 每个城市每个等级完成的任务数量。
    #     """
    #     # 将 Y_best_allocation 转换为 NumPy 数组
    #     Y_array = np.array(Y_best_allocation)

    #     # 从 Y_array 中提取城市和等级信息
    #     cities = Y_array[:, 0] - 1  # 城市索引从 0 开始
    #     levels = Y_array[:, 1] - 1  # 等级索引从 0 开始

    #     # 使用 NumPy 的 bincount 函数统计每个城市每个等级的任务数量
    #     n_il = np.bincount(cities * self.L_levels + levels, minlength=self.I_citys * self.L_levels).reshape(self.I_citys, self.L_levels)
    #     return n_il
    def count_tasks(self, Y_best_allocation):
        """
        统计决策Y中完成的任务数量。

        参数:
        Y_best_allocation (list): 决策。tuple (m, i, l) # m, i[1,I_citys] l[0,L_max]

        返回:
        np.array: 每个城市每个等级完成的任务数量。
        """
        # 将 Y_best_allocation 转换为 NumPy 数组
        Y_array = np.array(Y_best_allocation)

        # 从 Y_array 中提取城市和等级信息
        cities = Y_array[:, 1] - 1  # 城市索引从 0 开始
        levels = Y_array[:, 2]  # 等级索引从 0 开始

        # 创建一个零数组，形状为 (self.I_citys, self.L_levels)
        counts = np.zeros((self.I_citys, self.L_levels), dtype=int)

        # 统计任务数量
        for i in range(len(Y_best_allocation)):
            city = cities[i]
            level = levels[i]
            if level!=0: # 去除0 表示 业务员不分配的情况
                counts[city, level-1] += 1
        return counts


    def update_servers_info(self, servers_info, y, M_servers, I_citys, L_max):
        new_servers_info = []
        for m in range(M_servers):
            for i in range(I_citys):
                for l in range(L_max+1):
                    # print(f"{y[m, i, l]=}")
                    # city_assigned = pulp.value(y[m, i, l])
                    city_assigned = pulp.value(y[m, i, l])
                    if city_assigned == 1:
                        new_work_days  = servers_info[m][1] - 1
                        if l == 0:
                            if new_work_days >= 0:
                                new_servers_info.append((H_home_of_server[m], new_work_days))
                            else:
                                new_servers_info.append((H_home_of_server[m], W_workdays))
                        elif l != 0:
                            if new_work_days >= 0:
                                new_servers_info.append((i+1, new_work_days))
                            else:
                                new_servers_info.append((i+1, W_workdays))

                    # else:
                    #     new_servers_info.append(servers_info[m])
        
        return new_servers_info

    def func3_with_V_test(self):
        self.V = ValueFunction()
        self.pi = {}
        print(f"{self.max_iter=}")
        for i in range(self.max_iter):
            # 生成初始状态
            S = func1(I_citys=self.I_citys,
                    L_levels=self.L_levels,
                    W_workdays=self.W_workdays,
                    M_servers=self.M_servers,
                    x_max_task_num=self.x_max_task_num,
                    H_home_of_server=self.H_home_of_server,
                    lambd=self.lambd)

            # 初始化轨迹
            trajectory = []

            for t in range(self.T):
                print(f"{i=} {t=}")
                Xi_arriving_tasks = self.arriving_tasks_for_T[t]
                

                # 获取最优决策和收益
                # S, t, L_mathscr, H_home_of_server, r1_reward_for_level, c1_city_cost, c2, V, Xi_arriving_tasks
                Y_best_allocation, R_total_reward = self.func3_with_V(S=S,t=t,
                                                                    L_mathscr=self.mathscr_L,
                                                                    H_home_of_server=self.H_home_of_server,
                                                                    r1_reward_for_level=self.r1_reward_for_level,
                                                                    c1_city_cost=self.c1_city_cost,
                                                                    c2=self.c2,
                                                                    V=self.V,
                                                                    Xi_arriving_tasks=Xi_arriving_tasks)
                print("Y_best_allocation, R_total_reward" ,Y_best_allocation, R_total_reward)
            #     break
            # break
        self.V.save()

In [92]:
# 创建 Func8 类的实例
func8_instance = Func8()

func8_instance.run()

-------------迭代次数-i=0--------------
H_home_of_server=[21, 4, 1, 24, 9, 8, 8, 5, 24, 4, 22, 24, 18, 3, 19, 14, 2, 1, 3, 7, 8, 17, 20, 1, 18, 7, 23, 21, 23, 18, 14, 8, 15, 19, 9, 26, 1, 25, 26, 6]
S1_servers=[(21, 6), (4, 6), (17, 2), (15, 2), (17, 0), (6, 2), (18, 0), (2, 3), (18, 3), (1, 3), (3, 2), (20, 4), (22, 4), (5, 5), (11, 3), (22, 3), (21, 0), (22, 4), (3, 3), (22, 4), (11, 2), (20, 4), (7, 4), (16, 0), (1, 5), (13, 2), (2, 0), (13, 0), (9, 4), (5, 4), (14, 6), (7, 5), (16, 5), (19, 6), (2, 5), (5, 2), (14, 3), (6, 2), (4, 2), (2, 0)]
--------------t=0--------------
M_servers=40 I_citys=26 L_max=5
--------------t=1--------------
M_servers=40 I_citys=26 L_max=5
--------------t=2--------------
M_servers=40 I_citys=26 L_max=5
--------------t=3--------------
M_servers=40 I_citys=26 L_max=5
--------------t=4--------------
M_servers=40 I_citys=26 L_max=5
--------------t=5--------------
M_servers=40 I_citys=26 L_max=5
--------------t=6--------------
M_servers=40 I_citys=26 L_max=5
---

KeyboardInterrupt: 

In [ ]:


        # # 压缩后，参与计算调用func5_bar(), 
        # # 1. 通过规划算法计算得到一个决策，根据决策获取下一个状态
        # # 2. 将下一个状态进行压缩，重新参与规划算法的决策（这个是迭代过程实现的还是在单步过程计算两次规划算法?)
        # # S00->V00->S10->V10->S20
        # # S01->V01->S11->V11->S21
        # # S01计算时使用与S11状态相同的历史迭代遇到过的决策收益V10?

        # barS=aggreg_state(S1, Z_cluster_num, X, M_servers, I_citys, L_levels)

        # for j in range(max_iter):
        #     S = func1(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd)  # 随机生成初始状态
        #     SS, AA, RR = [], [], []  # 记录状态、决策和收益的序列
        #     for t in range(1, W_workdays+1):  # 对每个时间段
        #         A = None
        #         max_Q = float('-inf')
                
        #         for Y_best_allocation in func6(S, mathcal_L, mathscr_L, N_1, N_2):  # 遍历决策空间
        #             R_total_reward = func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1_reward_for_level, c1_city_cost, c2)[1]  # 计算即时收益
        #             next_S = self.transition(S, Y_best_allocation, func7)  # 计算下一个状态
        #             next_S_bar = func2(next_S, Z_cluster_num, X, M_servers, I_citys, L_levels)  # 状态压缩
        #             Q = R_total_reward + V_value_func[t][hash(next_S_bar)]  # 计算动作值函数
        #             if Q > max_Q:
        #                 max_Q = Q
        #                 A = Y_best_allocation

        #         SS.append(S)
        #         AA.append(A)
        #         RR.append(func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1_reward_for_level, c1_city_cost, c2)[1])

        #         S = self.transition(S, A, func7)  # 状态转移

        #     # 更新值函数
        #     G = 0
        #     for t in range(W_workdays-1, -1, -1):
        #         G += RR[t]  # 计算累积收益
        #         S_bar = func2(SS[t], Z_cluster_num, X, M_servers, I_citys, L_levels)  # 状态压缩
        #         n = self.count_accur(S_bar, SS[:t])  # 计算状态出现次数
        #         V_value_func[t][hash(S_bar)] = (G + (n-1) * V_value_func[t][hash(S_bar)]) / n  # 更新值函数

        # # 计算最优策略
        # for S in self.get_all_states(I_citys, L_levels, W_workdays, M_servers, x_max_task_num):
        #     pi[hash(S)] = None
        #     max_Q = float('-inf')
        #     for Y_best_allocation in func6(S, mathcal_L, mathscr_L, N_1, N_2):
        #         R_total_reward = func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1_reward_for_level, c1_city_cost, c2)[1]
        #         next_S = self.transition(S, Y_best_allocation, func7)
        #         next_S_bar = func2(next_S, Z_cluster_num, X, M_servers, I_citys, L_levels)
        #         Q = R_total_reward + V_value_func[W_workdays-1][hash(next_S_bar)]
        #         if Q > max_Q:
        #             max_Q = Q
        #             pi[hash(S)] = Y_best_allocation

        # return V_value_func, pi

In [15]:
# 示例参数
T = 7  # 时间步数量
x_max_task_num = 3  # 最大值
I_citys = 26  # 城市数量
L_levels = 5  # 等级数量

np.random.seed(42)
lambda_il = np.random.rand(I_citys, L_levels)  # 生成率参数矩阵

# 生成 arriving_tasks_i
arriving_tasks_for_T = func7(T, x_max_task_num, lambda_il)
print(arriving_tasks_for_T.shape, arriving_tasks_for_T)

(7, 26, 5) [[[0 0 0 0 1]
  [0 0 0 2 1]
  [0 0 1 0 0]
  [1 0 0 1 0]
  [1 0 0 0 0]
  [1 1 0 0 0]
  [1 0 0 1 1]
  [1 0 0 2 1]
  [0 0 0 0 2]
  [0 0 2 1 0]
  [0 2 0 1 0]
  [1 0 0 0 0]
  [0 0 2 0 0]
  [0 0 0 1 3]
  [1 0 0 0 0]
  [1 3 0 1 2]
  [0 0 1 0 0]
  [1 1 2 2 1]
  [0 0 3 0 2]
  [2 0 0 0 0]
  [0 1 0 0 0]
  [1 0 1 0 0]
  [0 0 1 3 0]
  [1 0 0 0 2]
  [0 0 0 0 0]
  [1 0 2 0 0]]

 [[0 0 1 0 0]
  [0 0 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [2 0 0 0 0]
  [2 0 0 0 0]
  [0 2 0 0 0]
  [0 0 0 0 1]
  [0 0 0 2 0]
  [1 0 3 0 0]
  [0 2 2 2 0]
  [1 0 1 0 2]
  [0 0 0 0 0]
  [2 0 1 0 2]
  [0 0 0 1 0]
  [1 2 0 1 0]
  [3 0 1 0 1]
  [0 0 0 0 0]
  [0 1 2 0 1]
  [1 1 1 1 0]
  [1 1 0 1 1]
  [0 2 1 0 0]
  [0 0 0 3 0]
  [2 3 0 2 0]
  [0 2 0 0 2]
  [0 0 0 0 3]]

 [[2 0 0 0 1]
  [0 0 1 0 1]
  [0 2 2 0 0]
  [0 0 1 0 0]
  [0 0 0 0 1]
  [1 0 1 1 0]
  [0 0 0 2 1]
  [0 0 0 0 0]
  [0 0 0 2 0]
  [0 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 2]
  [0 1 0 0 0]
  [0 0 0 0 1]
  [0 0 0 1 0]
  [0 1 0 0 0]
  [0 0 0 0 1]
  [0 1 0 1 1]
  [0 

In [ ]:
#函数8
import numpy as np



for j in range(J):
    s0 = func1(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd)
    xi = func7(T, x_max_task_num, lambda_il)
    R_total_reward=[0]*T
    for t in range(T):
        action = func3(s0, L_mathscr, H_home_of_server, r1_reward_for_level, c1_city_cost, c2)
        R_total_reward[t] = reward(s0,action)
        s1 = state_trans(s0,action,xi[t])
        
        



def reward(st,at):
    


def state_trans(S0,act,xi):   #状态转移
    dic1 = {}
    for i, row in enumerate(S0):
        for j, value in enumerate(row):
            dic1[(i+1, j+1)] = value
    dic2 = {(x[1], x[2]): 1 for x in act if x[2] != 0}
    S_A_cell = {}
    for key in dic1:
        if key in dic2:
            S_A_cell[key] = dic1[key] - dic2[key]
        else:
            S_A_cell[key] = dic1[key]
    S_A = [[0] *len(S0[0]) for _ in range(len(S0))]
    for key, value in S_A_cell.items():
        S_A[key[0]-1][key[1]-1] = value
        
    result = np.add(S_A,xi)
    return result

In [55]:
import numpy as np

def state_trans(S0,act,xi):
    dic1 = {}
    for i, row in enumerate(S0):
        for j, value in enumerate(row):
            dic1[(i+1, j+1)] = value
    dic2 = {(x[1], x[2]): 1 for x in act if x[2] != 0}
    S_A_cell = {}
    for key in dic1:
        if key in dic2:
            S_A_cell[key] = dic1[key] - dic2[key]
        else:
            S_A_cell[key] = dic1[key]
    S_A = [[0] *len(S0[0]) for _ in range(len(S0))]
    for key, value in S_A_cell.items():
        S_A[key[0]-1][key[1]-1] = value
        
    result = np.add(S_A,xi)
    return result

In [56]:
S=state[0]
act = A
xi = arriving_tasks_for_T[0]
tra = state_trans(S,act,xi)
print(tra)

[[ 0  1  0  1  0]
 [ 0  2  0  3  0]
 [ 1  2  1  3  3]
 [ 0  0  3  0  2]
 [ 0  3  0  2  3]
 [ 0 -1  1  0  1]
 [-1  1  0  1  2]
 [ 1  1  2  2  0]
 [ 1  0  2  0  0]
 [-1  0  1  1  3]
 [ 2  0  0  2  1]
 [ 0  1  0  0  1]
 [ 0 -1  1  0  0]
 [ 0  1  1  3  0]
 [ 0  4  0  0  0]
 [ 1  0  0  0  0]
 [ 1  0  1  1  4]
 [ 3  0  0  3  2]
 [ 0  0  1  1  1]
 [-1  0  1  0  0]
 [ 1  0  0  3  0]
 [ 0  0 -1  2  0]
 [ 0  1  2  1  0]
 [ 3  0  0  0 -1]
 [ 0  1  2  1  1]
 [ 0  0  0  2  3]]
